## Animations

the `fvcomersemviz` package contains functionality to create animations from the timeseries, maps, and community composition plots. 


### Timeseries animations

plots a growing line over time within the selected time window. can be multi-line (`vars`,`regions`, `stations`) or a single line.
Depth selection works the same as the static timeseries plots




In [ ]:
#Setup

BASE_DIR = "/data/proteus1/scratch/yli/project/lake_erie/output_updated_river_var"
FILE_PATTERN = "erie_00??.nc"
FIG_DIR      = "/data/proteus1/scratch/moja/projects/Lake_Erie/fvcomersem-viz/examples/plots/"


STATIONS = [
    ("WE12", 41.90, -83.10),
    ("WE13", 41.80, -83.20),
]

REGIONS = [
    ("Central", {
        "shapefile": "../data/shapefiles/central_basin_single.shp"
    }),
    ("East", {
        "shapefile": "../data/shapefiles/east_basin_single.shp"
    }),
    ("West", {
        "shapefile": "../data/shapefiles/west_basin_single.shp"
    }),
]

GROUPS = {
    "DOC":   "R1_c + R2_c + R3_c + T1_30d_c + T2_30d_c",  # dissolved organic carbon (sum of pools)
    "phyto": ["P1_c", "P2_c", "P4_c", "P5_c"],            # total phytoplankton carbon (sum)
    "zoo":   ["Z4_c", "Z5_c", "Z6_c"],                    # total zooplankton carbon (sum)
    "chl":   "P1_Chl + P2_Chl + P4_Chl + P5_Chl",         # total chlorophyll (sum)
}

PLOT_STYLES = {
    "temp":   {"line_color": "lightblue", "cmap": "coolwarm"},
    "DOC":   {"line_color": "blue", "cmap": "viridis"},
    "chl":   {"line_color": "lightgreen", "cmap": "Greens", "vmin": 0.0, "vmax": 5.0},
    "phyto": {"line_color": "darkgreen","cmap": "YlGn"},
    "zoo":   {"line_color": "purple","cmap": "PuBu"},
}


from fvcomersemviz.io import load_from_base
from fvcomersemviz.utils import out_dir, file_prefix
from fvcomersemviz.plot import (
    hr, info, bullet, kv,
    try_register_progress_bar,
    list_files, summarize_files,
    plot_call,
    print_dataset_summary,
    ensure_paths_exist,
    sample_output_listing,
)

import matplotlib.pyplot as plt
from IPython.display import display

bullet("\nStations (name, lat, lon):")
for s in STATIONS:
    bullet(f"• {s}")

bullet("\nRegions provided:")
for name, spec in REGIONS:
    bullet(f"• {name}: {spec}")
ensure_paths_exist(REGIONS)

#  Discover files
info(" Discovering files")
files = list_files(BASE_DIR, FILE_PATTERN)
summarize_files(files)
if not files:
    print("\nNo files found. Exiting.")
    sys.exit(2)

#  Load dataset
info(" Loading dataset (this may be lazy if Dask is available)")
ds = load_from_base(BASE_DIR, FILE_PATTERN)
bullet("Dataset loaded. Summary:")
print_dataset_summary(ds)

# Where figures will go / filename prefix
out_folder = out_dir(BASE_DIR, FIG_DIR)
prefix = file_prefix(BASE_DIR)
kv("Figure folder", out_folder)
kv("Filename prefix", prefix)

In [ ]:


from fvcomersemviz.plots.animate import animate_timeseries
# def animate_timeseries(
#     ds: xr.Dataset,
#     *,
#     vars: Sequence[str],
#     groups: Optional[Dict[str, Any]],
#     scope: str,                                   # "domain" | "region" | "station"
#     regions: Optional[Sequence[Tuple[str, Dict[str, Any]]]] = None,
#                                                   # For scope="region": list of (region_name, spec_dict)
#                                                   #   spec_dict: {"shapefile": "..."} OR {"csv_boundary": "...", "lon_col": "...", "lat_col": "..."}
#     stations: Optional[Sequence[Tuple[str, float, float]]] = None,
#                                                   # For scope="station": list of (name, lat, lon) in WGS84
#     # --- time filters (any combination; applied before spatial ops) ---
#     months: Optional[Union[int, Sequence[int]]] = None,  # e.g., 7 or [6,7,8]
#     years: Optional[Union[int, Sequence[int]]]  = None,  # e.g., 2018 or [2018,2019]
#     start_date: Optional[str] = None,                    # "YYYY-MM-DD"
#     end_date: Optional[str]   = None,                    # "YYYY-MM-DD"
#     at_time: Optional[Any] = None,                       # NEW: single explicit instant; any pandas-parsable timestamp
#                                                          #      (e.g., "2018-06-10 12:00"); selects the nearest data time.
#                                                          #      Produces a one-frame GIF unless combined with other series/scope lines.
#     at_times: Optional[Sequence[Any]] = None,            # NEW: list of explicit instants; sequence of pandas-parsable timestamps
#                                                          #      (e.g., ["2018-06-01 00:00","2018-06-10 12:00", ...]).
#                                                          #      For each requested instant, the nearest dataset timestep is used.
#                                                          #      Takes precedence over `frequency` when provided.
#     time_method: str = "nearest",                        # NEW: method used when matching `at_time/at_times` to data times.
#                                                          #      Typically "nearest". Pandas-style options like "pad"/"backfill"
#                                                          #      are also accepted if your time index is monotonic.
#     frequency: Optional[str] = None,                     # NEW: user-friendly sampling cadence for frames when `at_*` is not set.
#                                                          #      One of: "hourly" | "daily" | "monthly".
#                                                          #      Internally mapped to pandas offsets: H / D / MS (month-start).
#                                                          #      Samples one representative (nearest) timestep per period bucket.
#     # --- vertical selection (applied before series extraction) ---
#     depth: Any = "surface",                              # "surface" | "bottom" | "depth_avg"
#                                                          # int -> sigma index (k)
#                                                          # float in [-1,0] -> sigma value (s)
#                                                          # other float or {"z_m": z} or ("z_m", z) -> absolute depth (m, negative downward)
#     # --- output + styling ---
#     base_dir: str = "",                                  # Used to form filename prefix
#     figures_root: str = "",                              # Root folder for saving GIFs (module subdir auto-added)
#     combine_by: Optional[str] = None,                    # None | "var" | "region" | "station"
#                                                          #   None      -> one GIF per (scope item × variable)
#                                                          #   "var"     -> one GIF per scope item; lines = variables
#                                                          #   "region"  -> scope="region": one GIF per variable; lines = regions
#                                                          #   "station" -> scope="station": one GIF per variable; lines = stations
#     linewidth: float = 1.8,                              # Line width in the animation
#     figsize: Tuple[int, int] = (10, 4),                  # (width, height) inches
#     dpi: int = 150,                                      # Render resolution for saved GIF
#     styles: Optional[Dict[str, Dict[str, Any]]] = None,  # Optional style map; e.g., {"temp":{"line_color":"lightblue"}}
#     verbose: bool = True,                                # Print progress / debug
# ) -> List[str]:
#     """
#     Create growing-line time-series **GIF animations** from FVCOM–ERSEM datasets.
#
#     Parameters
#     ----------
#     ds : xarray.Dataset
#         Model dataset already opened/combined across files.
#     vars : sequence of str
#         Names to plot. Each entry may be a native variable (e.g., "temp") or a
#         composite/group key resolvable via `groups` (e.g., "chl", "DOC").
#     groups : dict or None
#         Composite definitions; expressions or lists summed elementwise.
#     scope : {"domain","region","station"}
#         What to animate:
#           - "domain"  → spatial mean over entire mesh (area-weighted if `art1` present).
#           - "region"  → mask to polygon per region (nodes/elements inside).
#           - "station" → nearest *node* to each (name, lat, lon).
#     regions, stations :
#         Required only for their respective scopes (validated).
#
#     Time filters
#     ------------
#     `months`, `years`, and/or `start_date`–`end_date` may be combined. Omitted ⇒ full span.
#
#     Depth selection
#     ---------------
#     "surface" | "bottom" | "depth_avg" | sigma index/value | absolute z (meters; negative downward).
#     Absolute-z slices are done *per variable* using vertical coordinates ("z"/"z_nele").
#
#     Combining (multi-line animations)
#     ---------------------------------
#     combine_by=None:
#         One GIF per (scope item × variable).
#     combine_by="var":
#         One GIF per scope item, overlaying all variables as separate lines.
#     combine_by="region":
#         (scope="region") One GIF per variable, overlaying all regions as lines.
#     combine_by="station":
#         (scope="station") One GIF per variable, overlaying all stations as lines.
#
#     Styling
#     -------
#     `styles` can provide per-series hints (e.g., color) keyed by var/region/station label.
#     Only `line_color` is used currently; others follow Matplotlib defaults.
#
#     Returns
#     -------
#     List[str]
#         Full file paths to the saved GIF(s).
#
#     Output filenames
#     ----------------
#     <prefix>__<ScopeOrName>__<VarOrMulti>__<DepthTag>__<TimeLabel>__TimeseriesAnim[__CombinedByX].gif
#       - prefix      = basename(base_dir)
#       - ScopeOrName = Domain | Region_<name> | Station_<name> | "All" (for combined comparisons)
#       - VarOrMulti  = variable name or "multi" when combining by var
#       - DepthTag    = e.g., surface | bottom | zavg | sigma-0.7 | zm-10
#       - TimeLabel   = built from months/years/range, e.g., "Jul__2018" or "2018-04-01 to 2018-10-31"
#
#     Notes
#     -----
#     - Spatial means use `art1` when available; otherwise simple means.
#     - Region masks accept shapefile or CSV boundary; elements can be derived from node masks if `nv` exists.
#     - Station selection uses great-circle distance on WGS84; longitudes west are negative.
#     - Works with Dask-backed datasets; computation occurs during reduction and GIF encoding.
#
#     Examples

# ------------------------------------------------------------------
# 1) DOMAIN — combine_by='var': one GIF, multiple lines = variables
# ------------------------------------------------------------------
info("[animate] Domain (one animation, lines = vars)…")
anim = animate_timeseries(
    ds,
    vars=["temp", "DOC", "chl", "phyto", "zoo"],
    groups=GROUPS,
    scope="domain",
    years=2018,
    depth="surface",
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    combine_by="var",            # one animation for the domain; lines are variables
    styles=PLOT_STYLES,
    verbose=True,
)

# ------------------------------------------------------------------
# 1b) DOMAIN — no combining: one GIF per variable (classic behaviour)
# ------------------------------------------------------------------
info("[animate] Domain (separate per variable)…")
anim = animate_timeseries(
    ds,
    vars=["temp", "DOC", "chl", "phyto", "zoo"],
    groups=GROUPS,
    scope="domain",
    years=2018,
    depth="surface",
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    combine_by=None,             # one animation per variable
    styles=PLOT_STYLES,
    verbose=False,
)

# ------------------------------------------------------------------
# 2) REGIONS — combine_by='var': one GIF per region, lines = variables
# ------------------------------------------------------------------
info("[animate] Regions (per region, lines = vars)…")
anim = animate_timeseries(
    ds,
    vars=["chl", "phyto", "zoo"],
    groups=GROUPS,
    scope="region",
    regions=REGIONS,
    months=[6, 7, 8], years=2018,
    depth={"z_m": -10},          # 10 m below surface
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    combine_by="var",            # one animation per region; lines are variables
    styles=PLOT_STYLES,
    verbose=False,
)

# ------------------------------------------------------------------
# 2b) REGIONS — combine_by='region': one GIF per variable, lines = regions
# ------------------------------------------------------------------
info("[animate] Regions (per var, lines = regions)…")
anim = animate_timeseries(
    ds,
    vars=["chl", "phyto"],
    groups=GROUPS,
    scope="region",
    regions=REGIONS,
    years=2018,
    depth="surface",
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    combine_by="region",         # one animation per variable; lines are regions
    styles=PLOT_STYLES,
    verbose=False,
)

# ------------------------------------------------------------------
# 3) STATIONS — combine_by=None: one GIF per (station × variable)
# ------------------------------------------------------------------
info("[animate] Stations (separate per station × variable)…")
anim = animate_timeseries(
    ds,
    vars=["chl", "phyto"],
    groups=GROUPS,
    scope="station",
    stations=STATIONS,
    start_date="2018-04-01", end_date="2018-10-31",
    depth="depth_avg",
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    combine_by=None,             # one per variable per station
    styles=PLOT_STYLES,
    verbose=False,
)

# ------------------------------------------------------------------
# 3b) STATIONS — combine_by='station': one GIF per variable, lines = stations
# ------------------------------------------------------------------
info("[animate] Stations (per var, lines = stations)…")
anim = animate_timeseries(
    ds,
    vars=["chl", "phyto"],
    groups=GROUPS,
    scope="station",
    stations=STATIONS,
    start_date="2018-04-01", end_date="2018-10-31",
    depth="surface",
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    combine_by="station",        # one animation per variable; lines are stations
    styles=PLOT_STYLES,
    verbose=False,
)

print(" Timeseries animation examples completed. Animations saved under:", FIG_DIR)


from IPython.display import display, Image, Video, HTML

RUN_ROOT = Path(FIG_DIR) / Path(BASE_DIR).name          # <FIG_DIR>/<basename(BASE_DIR)>
ANIM_DIR = RUN_ROOT / "animate"                      # flat folder with animations

search_root = ANIM_DIR if ANIM_DIR.exists() else RUN_ROOT

# Non-recursive listing; only .gif and .mp4
anims = sorted(
    [p for p in search_root.iterdir() if p.suffix.lower() in {".gif", ".mp4"}],
    key=lambda p: p.stat().st_mtime
)

if not anims:
    print(f"No GIF/MP4 animations found under {search_root}")
else:
    print(f"Found {len(anims)} animation(s) under {search_root}\n")
    for p in anims:
        display(HTML(f"<div style='font-family:monospace; margin:0.25em 0;'>• {p.relative_to(RUN_ROOT)}</div>"))
        if p.suffix.lower() == ".gif":
            display(Image(filename=str(p), width=720))
        else:  # .mp4
            display(Video(filename=str(p), embed=True, width=720))



In [ ]:

from fvcomersemviz.plots.animate import animate_maps
# def animate_maps(
#     ds: xr.Dataset,
#     *,
#     variables: Sequence[str],                         # Names to plot as individual frames/series
#     scope: str = "domain",                            # "domain" | "region"
#     regions: Optional[Sequence[Tuple[str, Dict[str, Any]]]] = None,
#                                                       # For scope="region": list of (region_name, spec_dict)
#                                                       #   spec_dict: {"shapefile": "..."} OR {"csv_boundary": "...", "lon_col": "...", "lat_col": "..."}
#     # --- time filters (any combination; applied before spatial ops) ---
#     months: Optional[Union[int, Sequence[int]]] = None,# e.g., 7 or [6,7,8]
#     years: Optional[Union[int, Sequence[int]]]  = None,# e.g., 2018 or [2018,2019]
#     start_date: Optional[str] = None,                  # "YYYY-MM-DD"
#     end_date: Optional[str]   = None,                  # "YYYY-MM-DD"
#     # --- explicit instants (override cadence) ---
#     at_time: Optional[Any] = None,                     # Single pandas-parsable instant → one-frame map (unless multiple vars/regions)
#     at_times: Optional[Sequence[Any]] = None,          # List of instants; nearest dataset timestep used for each
#     time_method: str = "nearest",                      # Matching method for at_time/at_times ("nearest", "pad", "backfill", ...)
#     frequency: Optional[str] = None,                   # "hourly" | "daily" | "monthly" | None → sampled cadence when no at_* given
#     # --- vertical selection (applied before mapping) ---
#     depth: Any = "surface",                            # "surface" | "bottom" | "depth_avg"
#                                                       # int -> sigma index (k)
#                                                       # float in [-1,0] -> sigma value (s)
#                                                       # other float or {"z_m": z} or ("z_m", z) -> absolute depth (m, negative)
#     # --- output + styling ---
#     base_dir: str = "",                                # Used to form filename prefix
#     figures_root: str = "",                            # Root folder for saving GIFs/MP4s (module subdir auto-added)
#     groups: Optional[Dict[str, Any]] = None,           # Composite variable definitions (e.g., {"chl": ["diatChl","flagChl",...]})
#     cmap: str = "viridis",                             # Matplotlib colormap name
#     clim: Optional[Tuple[float, float]] = None,        # (vmin, vmax); overrides robust quantiles if provided
#     robust_q: Tuple[float, float] = (5, 95),           # Percentiles for robust limits when clim not set
#     shading: str = "gouraud",                          # Node-centered default; element-centered forces "flat"
#     grid_on: bool = False,                             # Overlay mesh edges/nodes
#     figsize: Tuple[float, float]] = (8, 6),            # (width, height) inches
#     dpi: int = 150,                                    # Render resolution
#     interval_ms: int = 100,                            # Frame delay for GIF writer (ms)
#     fps: int = 10,                                     # Frames per second for MP4
#     styles: Optional[Dict[str, Dict[str, Any]]] = None,# Per-var/region style hints (e.g., {"temp": {"vmin":..,"vmax":..,"cmap":"..."}})
#     verbose: bool = True,                              # Print progress / debug
# ) -> List[str]:
#     """
#     Create **animated maps** (GIF/MP4) from FVCOM–ERSEM (or similar) datasets.
#
#     Parameters
#     ----------
#     ds : xarray.Dataset
#         Model dataset already opened/combined across files.
#     variables : sequence of str
#         Variables to render. Each may be a native variable (e.g., "temp") or a composite
#         resolvable via `groups` (e.g., "chl").
#     scope : {"domain","region"}
#         Spatial scope for each frame:
#           - "domain" → full mesh domain.
#           - "region" → mask/clip to polygon(s) provided in `regions`.
#     regions :
#         Required when `scope="region"`. Provide a list of tuples:
#           (region_name, {"shapefile": "/path/to/shape.shp"})
#         or
#           (region_name, {"csv_boundary": "/path/to/pts.csv", "lon_col": "lon", "lat_col": "lat"}).
#
#     Time filters
#     ------------
#     `months`, `years`, and/or `start_date`–`end_date` may be combined. If all omitted,
#     the full dataset span is used.
#
#     Explicit instants vs cadence
#     ----------------------------
#     If `at_time` or `at_times` is provided, the nearest dataset times are used per instant
#     (with `time_method`). When absent, `frequency` ("hourly"/"daily"/"monthly") samples a
#     representative timestep per period bucket.
#
#     Depth selection
#     ---------------
#     "surface" | "bottom" | "depth_avg" | sigma index/value | absolute z (meters; negative downward).
#     Absolute-z slices are applied per variable using appropriate vertical coordinates.
#
#     Styling
#     -------
#     Use `cmap`/`clim` or `robust_q` to control color scaling. `styles` can override per
#     variable/region (e.g., vmin, vmax, cmap). `shading="gouraud"` is suited to node-centered
#     fields; element-centered data should use "flat". `grid_on=True` draws mesh overlays.
#
#     Output
#     ------
#     Files are written under `<figures_root>/<basename(base_dir)>/animate` if present,
#     otherwise under the run root. Both GIF and/or MP4 may be produced depending on writer.
#
#     Returns
#     -------
#     List[str]
#         Full file paths to the saved animation(s).
#
#     Output filenames
#     ----------------
#     <prefix>__<ScopeOrName>__<Var>__<DepthTag>__<TimeLabel>__MAP.gif
#       - prefix      = basename(base_dir)
#       - ScopeOrName = Domain | Region_<name>
#       - Var         = variable name
#       - DepthTag    = e.g., surface | bottom | zavg | sigma-0.7 | zm-10
#       - TimeLabel   = built from months/years/range or explicit instants
#
#     Notes
#     -----
#     - Works with Dask-backed datasets; computation occurs during slicing/encoding.
#     - Area-weighting (via `art1`) may be used internally when aggregating to elements.
#     - MP4 output uses `fps`; GIF respects `interval_ms`.
#     """


# ==========================================================================
# === ANIMATION EXAMPLES =========================================
# ==========================================================================

# 4) DOMAIN MAPS - daily frames for June 2018 at the surface
#    Uses robust color limits per time window unless overridden by MAP_STYLES or clim/norm.
print("[animate] domain map animation (hourly, surface, June 2018)")
animate_maps(
    ds,
    variables=["temp", "chl"],   # native variables or GROUPS keys both work
    scope="domain",
    months=6, years=2018,
    depth="surface",
    groups=GROUPS,
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    frequency="daily",          #  hourly | daily | monthly
    grid_on=True,                # draw mesh overlay
    styles=PLOT_STYLES,           # optional per-var map styling
    verbose=True,
)

# 4b) REGION MAPS  all avaiablle frames across JJA 2018 at 10 m below surface
print("[animate] region map animation (daily, z=10 m below surface, JJA 2018)")
animate_maps(
    ds,
    variables=["chl"],
    scope="region",
    regions=REGIONS,
    months=[6, 7, 8], years=2018,
    depth={"z_m": -10},          # absolute metres below surface (negative down)
    groups=GROUPS,
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    grid_on=False,
    styles=PLOT_STYLES,
    verbose=True,
)


# 4c) DOMAIN MAPS - monthly frames for 2018 bottom layer
print("[animate] domain map animation (monthly, bottom, 2018)")
animate_maps(
    ds,
    variables=["chl"],
    scope="domain",
    years=2018,
    depth="bottom",
    groups=GROUPS,
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    frequency="monthly",
    styles=PLOT_STYLES,
    verbose=True,
)

# 4d) DOMAIN MAPS - explicit instants
print("[animate] domain map animation (explicit instants)")
animate_maps(
    ds,
    variables=["temp"],
    scope="domain",
    depth="depth_avg",
    groups=GROUPS,
    base_dir=BASE_DIR, figures_root=FIG_DIR,
    at_times=["2018-06-01 00:00", "2018-06-10 12:00", "2018-06-20 00:00"],
    grid_on=True,
    styles=PLOT_STYLES,
    verbose=True,
)

print(" Maps animation examples completed. Animations saved under:", FIG_DIR)

from pathlib import Path
from IPython.display import display, Image, Video, HTML

RUN_ROOT = Path(FIG_DIR) / Path(BASE_DIR).name          # <FIG_DIR>/<basename(BASE_DIR)>
ANIM_DIR = RUN_ROOT / "animate"                         # flat folder with animations

search_root = ANIM_DIR if ANIM_DIR.exists() else RUN_ROOT

# Non-recursive listing; only .gif and .mp4, and filename contains "MAP" (case-insensitive)
anims = sorted(
    [
        p for p in search_root.iterdir()
        if p.suffix.lower() in {".gif", ".mp4"} and "map" in p.name.lower()
    ],
    key=lambda p: p.stat().st_mtime
)

if not anims:
    print(f"No GIF/MP4 animations with 'MAP' in the name found under {search_root}")
else:
    print(f"Found {len(anims)} animation(s) with 'MAP' in the name under {search_root}\n")
    for p in anims:
        display(HTML(f"<div style='font-family:monospace; margin:0.25em 0;'>• {p.relative_to(RUN_ROOT)}</div>"))
        if p.suffix.lower() == ".gif":
            display(Image(filename=str(p), width=720))
        else:  # .mp4
            display(Video(filename=str(p), embed=True, width=720))
